In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spotify-music-data-to-identify-the-moods/data_moods.csv


In [2]:
data=pd.read_csv('/kaggle/input/spotify-music-data-to-identify-the-moods/data_moods.csv')

In [6]:
data.sample(5)

,name,album,artist,id,release_date,popularity,length,danceability,acousticness,energy,instrumentalness,liveness,valence,loudness,speechiness,tempo,key,time_signature,mood
260,ID8,ID8,Dimitri Vangelis & Wyman,2jxfHzOYVLmGKejNRHvqoc,2020-03-06,51,197183,0.576,0.019800,0.948,0.000123,0.6830,0.1930,-4.251,0.0898,128.029,9,3,Happy
611,Voices,The Sickness (10th Anniversary Edition),Disturbed,47H4oc7Zkihwae7ST7F4zp,2010-03-12,54,191960,0.589,0.000116,0.983,0.002110,0.0628,0.4690,-2.521,0.1210,110.007,10,4,Energetic
659,Yam Yam,Yam Yam,No Vacation,1j56M8iOGH48yuKqj64qGK,2017-05-12,0,216038,0.445,0.319000,0.591,0.766000,0.1070,0.0774,-9.004,0.0316,137.864,7,4,Sad
405,Pain,A Deeper Understanding,The War On Drugs,59P1nrdEImkAKa1nyW9X2e,2017-08-25,60,330760,0.523,0.003870,0.833,0.000339,0.0841,0.4820,-4.374,0.0405,115.975,7,4,Sad
39,Auburn Sky,Auburn Sky,Toledo Rains,42BtxDRqFTbE6Jb6rd9QOZ,2020-03-10,48,162829,0.265,0.906000,0.035,0.890000,0.0932,0.0651,-26.502,0.0359,132.972,4,4,Calm


In [9]:
data.value_counts('mood')

mood
Sad          197
Calm         195
Energetic    154
Happy        140
Name: count, dtype: int64

In [10]:
data.shape


(686, 19)

In [17]:
X=data.iloc[:,5:18]
X.sample(5)

,popularity,length,danceability,acousticness,energy,instrumentalness,liveness,valence,loudness,speechiness,tempo,key,time_signature
438,34,256259,0.594,0.9880,0.06240,0.9280,0.109,0.0738,-19.409,0.0472,123.999,10,4
536,60,230573,0.879,0.0370,0.69300,0.1190,0.180,0.9660,-10.577,0.0460,133.231,11,4
164,51,237346,0.602,0.0992,0.92600,0.0449,0.241,0.5740,-1.620,0.0897,100.014,5,4
563,61,164866,0.524,0.0252,0.88100,0.0000,0.101,0.6200,-4.479,0.0468,101.677,5,4
564,37,198250,0.334,0.9440,0.00961,0.9010,0.105,0.0916,-27.240,0.0416,68.153,5,3


In [4]:
y=data.iloc[:,-1]
y.sample(3)

496          Sad
479          Sad
241    Energetic
Name: mood, dtype: object

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.preprocessing import LabelEncoder
le=LabelEncoder();
numeric_labels=le.fit_transform(y_train)
numeric_labels_test=le.fit_transform(y_test)

# Convert class vectors to binary class matrices (one-hot encoding)
num_classes = len(set(y_train))
y_train_one_hot = to_categorical(numeric_labels, num_classes=num_classes)
y_test_one_hot = to_categorical(numeric_labels_test, num_classes=num_classes)

model = Sequential()

# Input layer
model.add(Dense(units=64, activation='relu', input_shape=(X_train.shape[1],)))

# Hidden layers
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.5)) 
model.add(Dense(units=64, activation='relu'))

# Output layer
model.add(Dense(units=num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
accuracy = model.evaluate(X_test, y_test_one_hot)[1]
print(f"Test Accuracy: {accuracy}")


Epoch 1/10
14/14 [==============================] - 1s 20ms/step - loss: 1.3108 - accuracy: 0.3767 - val_loss: 1.1286 - val_accuracy: 0.6909
Epoch 2/10
14/14 [==============================] - 0s 5ms/step - loss: 1.0104 - accuracy: 0.6484 - val_loss: 0.8825 - val_accuracy: 0.6909
Epoch 3/10
14/14 [==============================] - 0s 5ms/step - loss: 0.7697 - accuracy: 0.7215 - val_loss: 0.6864 - val_accuracy: 0.7636
Epoch 4/10
14/14 [==============================] - 0s 5ms/step - loss: 0.6093 - accuracy: 0.7557 - val_loss: 0.5513 - val_accuracy: 0.8091
Epoch 5/10
14/14 [==============================] - 0s 5ms/step - loss: 0.5083 - accuracy: 0.8174 - val_loss: 0.4987 - val_accuracy: 0.8000
Epoch 6/10
14/14 [==============================] - 0s 5ms/step - loss: 0.4746 - accuracy: 0.7991 - val_loss: 0.4543 - val_accuracy: 0.8182
Epoch 7/10
14/14 [==============================] - 0s 4ms/step - loss: 0.4471 - accuracy: 0.8311 - val_loss: 0.4337 - val_accuracy: 0.8364
Epoch 8/10
14/14 [=

In [28]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

le = LabelEncoder()
numeric_labels_train = le.fit_transform(y_train)
numeric_labels_test = le.transform(y_test)

# Convert class vectors to binary class matrices (one-hot encoding)
num_classes = len(set(y_train))
y_train_one_hot = to_categorical(numeric_labels_train, num_classes=num_classes)
y_test_one_hot = to_categorical(numeric_labels_test, num_classes=num_classes)

model = Sequential()


model.add(Dense(units=64, activation='relu', input_shape=(X_train.shape[1],)))


model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.5))  
model.add(Dense(units=64, activation='relu'))


model.add(Dense(units=num_classes, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=80, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    X_train, y_train_one_hot,
    epochs=100, batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping]
)


accuracy = model.evaluate(X_test, y_test_one_hot)[1]
print(f"Test Accuracy: {accuracy}")


Epoch 1/100
14/14 [==============================] - 1s 19ms/step - loss: 1.2775 - accuracy: 0.4224 - val_loss: 1.0872 - val_accuracy: 0.7091
Epoch 2/100
14/14 [==============================] - 0s 5ms/step - loss: 1.0120 - accuracy: 0.6370 - val_loss: 0.8650 - val_accuracy: 0.7455
Epoch 3/100
14/14 [==============================] - 0s 6ms/step - loss: 0.7702 - accuracy: 0.7306 - val_loss: 0.6798 - val_accuracy: 0.7727
Epoch 4/100
14/14 [==============================] - 0s 5ms/step - loss: 0.6156 - accuracy: 0.7580 - val_loss: 0.5668 - val_accuracy: 0.7727
Epoch 5/100
14/14 [==============================] - 0s 5ms/step - loss: 0.5441 - accuracy: 0.7626 - val_loss: 0.5095 - val_accuracy: 0.7636
Epoch 6/100
14/14 [==============================] - 0s 6ms/step - loss: 0.4768 - accuracy: 0.8014 - val_loss: 0.4795 - val_accuracy: 0.7727
Epoch 7/100
14/14 [==============================] - 0s 5ms/step - loss: 0.4823 - accuracy: 0.7991 - val_loss: 0.4505 - val_accuracy: 0.8000
Epoch 8/100
